# ResNet-18 Inference example  

For the first time usage, make sure you have run all the cells in [Get-ResNet-18_Model.ipynb](./Get-ResNet-18_Model.ipynb)

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO
import numpy as np

In [ ]:

model = models.resnet18()
state_dict = torch.load('zoo/resnet18_model.pth')
model.load_state_dict(state_dict)
model.eval()

In [ ]:

import json
file_path = 'zoo/imagenet-simple-labels.json'
with open(file_path, 'r') as f:
    labels = json.load(f)


In [ ]:
# Perform inference on an example image
image_url = 'https://www.hartz.com/wp-content/uploads/2022/04/small-dog-owners-1.jpg'  # Provide an image URL
# Download the image
response = requests.get(image_url,verify=False)


In [ ]:

img = Image.open(BytesIO(response.content))


In [ ]:


# Preprocess the image
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_data = preprocess(img)
input_data = input_data.unsqueeze(0)  # Add batch dimension

# Perform inference
with torch.no_grad():
    output = model(input_data)

# Post-process the results
probabilities = torch.nn.functional.softmax(output[0], dim=0)
predicted_class = torch.argmax(probabilities).item()



# Get the predicted label
predicted_label = labels[predicted_class]

print(f"Predicted class: {predicted_class}")
print(f"Predicted label: {predicted_label}")
print(f"Probability: {probabilities[predicted_class]:.2%}")


In [ ]:
input_data.shape

In [ ]:
X = input_data.numpy()
print(X.shape)

In [ ]:
np.save("X",X)

## ! Channel first !

In [ ]:

X=np.load('X.npy')

In [ ]:
X.shape

## Test image for inference

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
#plt.figure()
axes[0].imshow(X[0][0], cmap='Reds')
axes[1].imshow(X[0][1], cmap='Greens')
axes[2].imshow(X[0][2], cmap='Blues')
#plt.colorbar()
plt.grid(False)
plt.show()



In [ ]:
X.dtype

In [ ]:
X_transposed = np.transpose(X[0], (1, 2, 0))

# Plot the image
plt.imshow(X_transposed)
plt.axis('off')  # Turn off axis labels
plt.show()

# ONNX format

In [ ]:
import os
import pathlib
import onnx
import netron
import numpy as np


In [ ]:

def show_model(model_file_name,itf='10.217.184.110',port=8098):
    netron.start(file=model_file_name,address=(itf,port))
    return port

In [ ]:
input_path=pathlib.Path("onnx/resnet18.onnx")
#input_path=pathlib.Path("onnx/submodel_46_48.onnx")
onnx_model = onnx.load(input_path)
onnx.checker.check_model(onnx_model)

In [ ]:
def get_opset(model):
    fields =model.opset_import
    field=  fields[0]
    return field.version

In [ ]:
f"{get_opset(onnx_model)}"

In [ ]:
port=show_model("./"+str(input_path))

## ONNX Inference

pip install onnxruntime

In [ ]:
graph = onnx_model.graph
input_name=graph.node[0].input[0]

In [ ]:
outputs=[]
for node in graph.node:
    outputs.append(node.output[0])
print(outputs)    

In [ ]:
outputs=['/Flatten_output_0','191']

import google.protobuf.internal.containers 
google.protobuf.internal.containers.RepeatedScalarFieldContainer: out =outputs[0]
type(out[0])

In [ ]:
import onnx.helper as helper
for tensor_name in outputs:
    intermediate_tensor_name = tensor_name
    intermediate_layer_value_info = helper.ValueInfoProto()
    intermediate_layer_value_info.name = intermediate_tensor_name
    onnx_model.graph.output.extend([intermediate_layer_value_info])

In [ ]:
import onnxruntime as rt
import numpy as np

sess = rt.InferenceSession(onnx_model.SerializeToString(),
                                        providers=["CPUExecutionProvider"])


In [ ]:
X = np.load('X.npy')
onnx_pred = sess.run(outputs, {input_name: X})
#X.dtype

In [ ]:
def softmax(x, axis=None):
    x = x - x.max(axis=axis, keepdims=True)
    y = np.exp(x)
    return y / y.sum(axis=axis, keepdims=True)

In [ ]:
last_idx=len(onnx_pred)-1
graph_output=onnx_pred[last_idx]
probabilities = softmax(graph_output[0])

In [ ]:
idxMax = np.argmax(graph_output)
print(f"Predicted class is '{idxMax}' with probability {probabilities[idxMax]}")  


In [ ]:
X1=onnx_pred[0]
np.save('X1',X1)

## Export input

In [ ]:
def export_to_c_array(var):
    theShape=var.shape;
    theRank=len(theShape)
    txt =f"{{\n"
    #print(f"the rank={theRank}, the shape={theShape}")
    if theRank == 1:
        
        txt =f"{{ {var[0]}"
        for i in range(1,theShape[0]):
             txt+=f", {var[i]}"
        txt+=f' }}'     
        #print(txt)
        return txt
        
    for i in range(0,theShape[0]):
        #print(f"-recursive {i},{var.shape}")
        a = var[i] 
        txt+=export_to_c_array(a)
        #txt+=f"}}\n"
        if i<theShape[0]-1:
            txt+=',\n'
        else:
            txt+=' \n'    
    return txt+f"}}"

def export_to_c(file_name,varname,var):
    txt = f"float {varname}"
    for k in var.shape:
        txt+=f"[{k}]"
    txt+= ' = \n'
    values =export_to_c_array(var)
    txt+= f"{values};"
    cpp_file = open(file_name, "w")
    cpp_file.write(txt)
    cpp_file.close()
    return txt

In [ ]:
echo=export_to_c('data.cpp','input',X)
print(echo)


## Save intermediary outputs

In [ ]:
i =1
for pred in onnx_pred:
    fname=f"y_{i}_ref"
    np.save(fname,pred)
    i+=1
    

# playground

In [ ]:
pip install pydot

In [ ]:
from onnx.tools.net_drawer import GetPydotGraph, GetOpNodeProducer

pydot_graph = GetPydotGraph(
    onnx_model.graph,  # model_onnx is a ModelProto instance
    name=onnx_model.graph.name,
    rankdir="TP",
    node_producer=GetOpNodeProducer("docstring"))
pydot_graph.write_dot("graph.dot")

In [ ]:
def generate_operators_report(onnx_model):
    operators_list = []
    operator_idx={}
    operators_occurence_dict = dict()

    idx =0
    for operator in onnx_model.graph.node:
        operator_idx.update({idx: [operator.op_type,operator.name,operator.output]})
        idx+=1
        operators_list.append(operator.op_type)
        if operator.op_type in operators_occurence_dict:
            operators_occurence_dict[operator.op_type] += 1
        else:
            operators_occurence_dict[operator.op_type] = 1

    sorted_operators_occurence_dict = dict(sorted(operators_occurence_dict.items(), key=lambda x: x[1], reverse=True))

    return operators_list, sorted_operators_occurence_dict,operator_idx

In [ ]:
op_list,sorted_op_list,op_id =generate_operators_report(onnx_model=onnx_model)

In [ ]:
op_id

In [ ]:
idx_start=0
idx_end=0


In [ ]:
inputs = [ onnx_model.graph.node[idx_start].input[0]]
outputs = onnx_model.graph.node[idx_end].output

In [ ]:
print(F"{inputs}==>{outputs}")

In [ ]:
old_model=str(input_path)


In [ ]:
new_model=f'onnx/submodel_{idx_start}_{idx_end}.onnx'


In [ ]:
onnx.utils.extract_model(old_model, new_model, inputs,outputs) 


In [ ]:
onnx_new_model = onnx.load(new_model)
onnx.checker.check_model(onnx_new_model)

In [ ]:
port=show_model("./"+str(new_model))

In [ ]:
import onnxruntime as rt
sess = rt.InferenceSession(onnx_new_model.SerializeToString(),
                                        providers=["CPUExecutionProvider"])

In [ ]:
X = np.load('X.npy')
#X = np.load('X1.npy')
#X = np.load('X2.npy')
input_name=onnx_new_model.graph.node[0].input[0]
onnx_pred = sess.run(outputs, {input_name: X})

In [ ]:
last_idx=len(onnx_pred)-1
graph_output=onnx_pred[last_idx]

In [ ]:
graph_output.shape

In [ ]:
probabilities = softmax(graph_output[0])
idxMax = np.argmax(graph_output)
print(f"Predicted class is '{idxMax}' with probability {probabilities[idxMax]}") 

In [ ]:
echo=export_to_c('data2.cpp','input',X)

In [ ]:
np.save(f"Y_{idx_start}_{idx_end}",graph_output)

In [ ]:
myTensor=np.array([
[
[	[1.,2,3],
    [4,5,7],
    [8,9,10],
    [81,91,11]
],
[	[10.,20,30],
    [40,50,70],
    [80,90,10],
    [801,901,101]
]
]
],dtype= np.float32)

print(export_to_c_array(myTensor))